# Ingredient Embeddings
BS"D

In this notebook, we will create embeddings for ingredients in the dataset. We will first attempt to use the gensim library.

In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from gensim.models import Word2Vec

ModuleNotFoundError: No module named 'gensim'

## Load Data
We have two datasets, labeled `dataset_1.json` and `dataset_2.json`. We will initially only use `dataset_2.json` for the embeddings since it is seemingly more precise.

In [10]:
filepath = 'data/dataset_1.json'

recipes = pd.read_json(filepath, orient='table')

recipes

,ingredients
0,"[whole chicken, kosher salt, acorn squash, uns..."
1,"[egg white, new potato, kosher salt, pepper]"
2,"[evaporated milk, whole milk, garlic powder, o..."
3,"[round, loaf, olive oil, sausage, unsalted but..."
4,"[dark brown sugar, hot water, fresh lemon juic..."
...,...
13496,"[all-purpose flour, unsweetened cocoa powder, ..."
13497,"[lemon, squash, olive oil, onion, couscous, ac..."
13498,"[katsuo bushi, dried bonito flake, dashi, sake..."
13499,"[unsalted butter, baby spinach, phyllo]"


## Train Embeddings